In [1]:
# Import modules from llama_index and langchain
from llama_index import SimpleDirectoryReader, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import os
from IPython.display import Markdown, display

In [2]:
# Adapted based on LlamaIndex documentation https://gpt-index.readthedocs.io/en/latest/index.html
# and Dan Shipper's work https://www.lennysnewsletter.com/p/i-built-a-lenny-chatbot-using-gpt

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 256
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define LLM (ChatGPT gpt-3.5-turbo)
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index.json')

    return index


def ask_me_anything(question):

    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    response = index.query(question, response_mode="compact")

    display(Markdown(f"You asked: <b>{question}</b>"))
    display(Markdown(f"Bot says: <b>{response.response}</b>"))

In [3]:
openai_key = open('openai_key.txt', 'r').read()

os.environ["OPENAI_API_KEY"] = openai_key

In [7]:
construct_index('textdata')

INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total embedding token usage: 68825 tokens


In [13]:
ask_me_anything('Proteína faz mal aos rins?')

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 724 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens


You asked: <b>Proteína faz mal aos rins?</b>

Bot says: <b>
Não, a proteína não faz mal aos rins. No entanto, é importante consumir proteína de forma moderada para evitar problemas de saúde. Além disso, é importante consultar um médico para obter orientações específicas sobre o consumo de proteína, dependendo de suas condições de saúde.</b>